In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler
import torch
from torch.nn.functional import softmax
import datasets
from datasets import load_dataset, Dataset
import random
from torch.utils.data import DataLoader, Subset
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchmetrics import F1Score
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
pd.options.mode.copy_on_write = True

In [3]:
fpath = "/Users/barrychen/Desktop/IELTSWritingHelper/datasets_ready/Lexical.csv"
df4 = pd.read_csv(fpath)
# df4['score'] = df4['score'].round(1)
# df_filtered4 = df4[(df4['score'] > 1.5) & (df4['score'] < 12.0)]

# reverse_mapping_3 = {
#     3.5: 0, 4.0: 0,
#     4.5: 1, 5.0: 1,
#     5.5: 2, 6.0: 2,
#     6.5: 3, 7.0: 3,
#     7.5: 4, 8.0: 4,
#     8.5: 5, 9.0: 5
# }

# df_filtered4['score'] = df_filtered4['score'].map(reverse_mapping_3)

# df_sampled4 = df_filtered4.groupby('score').sample(
#     n=290, 
#     random_state=42
# ).reset_index(drop=True)

# df_sampled4['score'] = df_sampled4['score'].astype(int)

In [4]:
value_counts_df = df4["score"].value_counts().reset_index()
value_counts_df.columns = ["score", "count"]
value_counts_df = value_counts_df.sort_values(by="score", ascending=False).reset_index(drop=True)
value_counts_df

,score,count
0,10000.00,1
1,7117.00,1
2,2019.00,1
3,2018.00,1
4,2012.00,1
5,2000.00,1
6,1319.00,1
7,100.00,1
8,80.00,1
9,60.00,1


In [5]:
def df_sampling(fpath):
    df = pd.read_csv(fpath)
    df['score'] = df['score'].round(1)
    df_filtered = df[df['score'].isin([3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0])]
    reverse_mapping = {
        3.5: 0, 4.0: 0,
        4.5: 1, 5.0: 1,
        5.5: 2, 6.0: 2,
        6.5: 3, 7.0: 3,
        7.5: 4, 8.0: 4,
        8.5: 5, 9.0: 5
    }
    df_filtered['score'] = df_filtered['score'].map(reverse_mapping)

    # n = 50
    # df_sampled = df_filtered.groupby('score').sample(
    #     n=n, 
    #     random_state=42
    #     ).reset_index(drop=True)

    df_sampled = df_filtered.groupby('score', group_keys=False).apply(
        lambda x: x.sample(len(x), random_state=42)
        ).reset_index(drop=True)
    
    df_sampled['score'] = df_sampled['score'].astype(int)

    return df_sampled

fpath_ta = "/Users/barrychen/Desktop/IELTSWritingHelper/datasets_ready/Task_Achievement.csv"
fpath_co = "/Users/barrychen/Desktop/IELTSWritingHelper/datasets_ready/Coherence.csv"
fpath_le = "/Users/barrychen/Desktop/IELTSWritingHelper/datasets_ready/Lexical.csv"
fpath_gr = "/Users/barrychen/Desktop/IELTSWritingHelper/datasets_ready/Grammatical.csv"

df_ta = df_sampling(fpath_ta)
df_co = df_sampling(fpath_co)
df_le = df_sampling(fpath_le)
df_gr = df_sampling(fpath_gr)

/var/folders/vx/57ntql9x3lg7fsldp3xp6w1c0000gn/T/ipykernel_13740/4189222091.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df_filtered.groupby('score', group_keys=False).apply(
/var/folders/vx/57ntql9x3lg7fsldp3xp6w1c0000gn/T/ipykernel_13740/4189222091.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df_filtered.groupby('score', group_keys=False).apply(
/var/folders/vx/57

In [9]:
df_ta.head()

,prompt,essay,text,score
0,Some people say that to prevent illness and di...,Emerging disease is a complex matter as it IS ...,The candidate has effectively addressed the gi...,0
1,Nowadays celebrities are more famous for their...,"In this present world, famous personalities a...",The essay adequately addresses the task and at...,0
2,More people decided to have children in their ...,"currently, there are more and more people make...",The essay generally addresses the task by disc...,0
3,Many people believe that the current system of...,Once every month no private vehicles a day ca...,The essay fails to address the prompt effectiv...,0
4,Some people think that instead of preventing c...,"In today's time, human activities are having a...",The essay effectively addresses the given task...,0


In [8]:
# Specify the file name from which to load the model
modelsavename = "deberta_9k_ep15_4597.pt"

# Initialize the same model architecture
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis",num_labels=6, ignore_mismatched_sizes=True)

# Load the saved state_dict into the model
with open(modelsavename, "rb") as f:
    model.load_state_dict(torch.load(f))

# Move model to the device (GPU if available)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

tokenizer = AutoTokenizer.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis")

print(f"Model loaded from {modelsavename}")

/Users/barrychen/anaconda3/envs/ece1786/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at mrm8488/deberta-v3-ft-financial-news-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/vx/57ntql9x3lg7fsldp3xp6w1c0000gn/T/ipykernel_13740/2724072977.py:9: FutureWarning: You are using `torch.load` w

Model loaded from deberta_9k_ep15_4597.pt


In [11]:
def get_class(input_text):
  # Tokenize input and get model output
  inputs = tokenizer(input_text, return_tensors="pt")

  # Get logits (output for classification)
  outputs = model(**inputs)
  logits = outputs.logits

  # Convert logits to probabilities using softmax
#   probs = softmax(logits, dim=1)

  # Map classes to polarity values
#   polarity_values = torch.tensor([-1.0, 0.0, 1.0])

  # Calculate polarity score as the weighted sum of probabilities
#   polarity_score = torch.sum(probs * polarity_values, dim=1).item()

  # Get the predicted class (optional, for reference)
  predicted_class = logits.argmax(dim=1).item()

  return predicted_class

def compute_mae(df):
    df['score_pred'] = df.apply(lambda row: get_class(row['prompt'] + row['essay'] + row['text']), axis=1)
    mae = mean_absolute_error(df['score'], df['score_pred'])
    return mae

In [12]:
mae_ta = compute_mae(df_ta)
print('MAE for task achievement is', mae_ta)

MAE for task achievement is 0.5481417083242773


In [13]:
# Specify the file name from which to load the model
modelsavename_co = "COHERENCE_EPOCH4_F10.4253.pt"

# Initialize the same model architecture
model_co = AutoModelForSequenceClassification.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis",num_labels=6, ignore_mismatched_sizes=True)

# Load the saved state_dict into the model
with open(modelsavename_co, "rb") as f:
    model_co.load_state_dict(torch.load(f))

# Move model to the device (GPU if available)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_co.to(device)

tokenizer = AutoTokenizer.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis")

print(f"Model loaded from {modelsavename_co}")

/Users/barrychen/anaconda3/envs/ece1786/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at mrm8488/deberta-v3-ft-financial-news-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/vx/57ntql9x3lg7fsldp3xp6w1c0000gn/T/ipykernel_13740/688501456.py:9: FutureWarning: You are using `torch.load` wi

Model loaded from COHERENCE_EPOCH4_F10.4253.pt


In [14]:
mae_co = compute_mae(df_co)
print('MAE for coherence is', mae_co)

MAE for coherence is 0.6632443531827515


In [15]:
# Specify the file name from which to load the model
modelsavename_le = "Lexical_epoch5.pt"

# Initialize the same model architecture
model_le = AutoModelForSequenceClassification.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis",num_labels=6, ignore_mismatched_sizes=True)

# Load the saved state_dict into the model
with open(modelsavename_le, "rb") as f:
    model_le.load_state_dict(torch.load(f))

# Move model to the device (GPU if available)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_le.to(device)

tokenizer = AutoTokenizer.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis")

print(f"Model loaded from {modelsavename_le}")

/Users/barrychen/anaconda3/envs/ece1786/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at mrm8488/deberta-v3-ft-financial-news-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/vx/57ntql9x3lg7fsldp3xp6w1c0000gn/T/ipykernel_13740/934866803.py:9: FutureWarning: You are using `torch.load` wi

Model loaded from Lexical_epoch5.pt


In [16]:
mae_le = compute_mae(df_le)
print('MAE for lexical is', mae_le)

MAE for lexical is 0.6453399316601862


In [17]:
# Specify the file name from which to load the model
modelsavename_gr = "grammatical_epoch4_F10.5099.pt"

# Initialize the same model architecture
model_gr = AutoModelForSequenceClassification.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis",num_labels=6, ignore_mismatched_sizes=True)

# Load the saved state_dict into the model
with open(modelsavename_gr, "rb") as f:
    model_gr.load_state_dict(torch.load(f))

# Move model to the device (GPU if available)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_gr.to(device)

tokenizer = AutoTokenizer.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis")

print(f"Model loaded from {modelsavename_gr}")

/Users/barrychen/anaconda3/envs/ece1786/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at mrm8488/deberta-v3-ft-financial-news-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/vx/57ntql9x3lg7fsldp3xp6w1c0000gn/T/ipykernel_13740/3790183536.py:9: FutureWarning: You are using `torch.load` w

Model loaded from grammatical_epoch4_F10.5099.pt


In [18]:
mae_gr = compute_mae(df_gr)
print('MAE for grammatical is', mae_gr)

MAE for grammatical is 0.6624760536398467
